In [1]:
# ====== [1] Imports and Setup ======
import os
import re
import string
import shutil
import pandas as pd
import numpy as np
import joblib
import emoji
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

from transformers import (
    DistilBertTokenizerFast, DistilBertForSequenceClassification,
    BertTokenizerFast, BertForSequenceClassification,
    RobertaTokenizerFast, RobertaForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback
)
from datasets import Dataset

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\12601\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\12601\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\12601\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
c:\Users\12601\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ====== [2] Directory Setup ======
BASE_DIR = os.path.join(".", "saved_models")
TRADITIONAL_DIR = os.path.join(BASE_DIR, "traditional")
TRANSFORMERS_DIR = os.path.join(BASE_DIR, "transformers")
os.makedirs(TRADITIONAL_DIR, exist_ok=True)
os.makedirs(TRANSFORMERS_DIR, exist_ok=True)

# Clear Hugging Face cache to avoid corrupted/incomplete models
hf_cache_dir = os.path.expanduser("~/.cache/huggingface/transformers")
if os.path.exists(hf_cache_dir):
    print("\n🧹 Clearing Hugging Face Transformers cache...")
    shutil.rmtree(hf_cache_dir)
    print("✅ Cache cleared successfully.\n")

In [4]:
# ====== [3] Emoticon Dictionary ======
emoticon_dict = {
    ":)": "smiley", ":-)": "smiley", ":D": "laugh", ":-D": "laugh",
    ":(": "sad", ":-(": "sad", ":'(": "cry", ":'-(": "cry",
    ":P": "playful", ":-P": "playful", ";)": "wink", ";-)": "wink",
    ":/": "skeptical", ":-/": "skeptical", ":|": "neutral", ":-|": "neutral",
    ":O": "surprised", ":-O": "surprised", "XD": "laugh", "<3": "love",
    ">:(": "angry", "D:": "horrified", ":-*": "kiss", ":3": "cute",
    ":-X": "sealed lips", "B-)": "cool", "O:)": "angel", ">:)": "evil smile"
}

In [5]:
# ====== [4] Load Dataset ======
train_df = pd.read_csv("C:/Users/12601/OneDrive/Desktop/collage works/mini prj/train.csv")
test_df = pd.read_csv("C:/Users/12601/OneDrive/Desktop/collage works/mini prj/test.csv")

train_df = train_df[train_df["sentiment"] != "neutral"]
test_df = test_df[test_df["sentiment"] != "neutral"]

label_map = {"negative": 0, "positive": 1}
train_df["label"] = train_df["sentiment"].map(label_map)
test_df["label"] = test_df["sentiment"].map(label_map)

In [6]:
# ====== [5] Preprocessing Function ======
def preprocess(text):
    text = text.lower()
    text = re.sub(r"http\\S+", "link", text)
    text = re.sub(r"@\\w+", "user_mention", text)
    text = re.sub(r"#(\\w+)", lambda m: m.group(1).replace("_", " "), text)
    text = re.sub(r"\\d+", "", text)

    for emot, meaning in emoticon_dict.items():
        text = text.replace(emot, f" {meaning} ")

    text = emoji.demojize(text)
    text = text.replace(":", "").replace("_", " ")
    text = re.sub(rf"[{string.punctuation}]", "", text)
    text = re.sub(r"(.)\\1{2,}", r"\\1\\1", text)

    tokens = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    stop_words.difference_update({"no", "not", "don", "shouldn", "wasn", "mustn"})
    tokens = [w for w in tokens if w not in stop_words]

    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(lemmatizer.lemmatize(w)) for w in tokens]
    return " ".join(tokens)

train_df["clean_text"] = train_df["text"].apply(preprocess)
test_df["clean_text"] = test_df["text"].apply(preprocess)

In [7]:
# ====== [6] Feature Extraction and Traditional Model Training ======
vectorizers = {
    "count_bi": CountVectorizer(ngram_range=(1, 2)),
    "tfidf_bi": TfidfVectorizer(ngram_range=(1, 2))
}

X_train_texts = train_df["clean_text"]
y_train = train_df["label"]
X_test_texts = test_df["clean_text"]
y_test = test_df["label"]

for name, vectorizer in vectorizers.items():
    X_train = vectorizer.fit_transform(X_train_texts)
    X_test = vectorizer.transform(X_test_texts)

    for model_name, model in {
        "nb": MultinomialNB(),
        "lr": LogisticRegression(max_iter=1000),
        "svm": SVC(kernel="linear", probability=True)
    }.items():
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        print(f"\n=== {model_name.upper()} with {name} ===")
        print("Accuracy:", accuracy_score(y_test, preds))
        print(classification_report(y_test, preds))
        joblib.dump((model, vectorizer), os.path.join(TRADITIONAL_DIR, f"{model_name}_{name}.joblib"))


=== NB with count_bi ===
Accuracy: 0.8944866920152091
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      1001
           1       0.89      0.91      0.90      1103

    accuracy                           0.89      2104
   macro avg       0.89      0.89      0.89      2104
weighted avg       0.89      0.89      0.89      2104


=== LR with count_bi ===
Accuracy: 0.9011406844106464
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      1001
           1       0.92      0.88      0.90      1103

    accuracy                           0.90      2104
   macro avg       0.90      0.90      0.90      2104
weighted avg       0.90      0.90      0.90      2104


=== SVM with count_bi ===
Accuracy: 0.8968631178707225
              precision    recall  f1-score   support

           0       0.88      0.90      0.89      1001
           1       0.91      0.89      0.90      1103

    accuracy    

In [13]:
# ====== [7] Universal Transformer Trainer ======
def train_transformer(model_cls, tokenizer_cls, model_name, train_texts, test_texts, y_train, y_test):
    print(f"\n🚀 Training {model_name}...")
    tokenizer = tokenizer_cls.from_pretrained(model_name)

    def tokenize(batch):
        return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

    train_ds = Dataset.from_pandas(pd.DataFrame({"text": train_texts, "label": y_train}))
    test_ds = Dataset.from_pandas(pd.DataFrame({"text": test_texts, "label": y_test}))
    train_ds = train_ds.map(tokenize, batched=True).remove_columns(["text"])
    test_ds = test_ds.map(tokenize, batched=True).remove_columns(["text"])

    model = model_cls.from_pretrained(model_name, num_labels=2)
    output_path = os.path.join(TRANSFORMERS_DIR, model_name.replace("/", "_"))

    is_large = "large" in model_name
    if is_large:
        args = TrainingArguments(
            output_dir=output_path,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            logging_dir=os.path.join(output_path, "logs"),
            per_device_train_batch_size=4,
            per_device_eval_batch_size=4,
            gradient_accumulation_steps=4,
            num_train_epochs=2,
            learning_rate=2e-5,
            weight_decay=0.01,
            warmup_steps=100,
            load_best_model_at_end=True,
            save_total_limit=1,
            metric_for_best_model="accuracy",
            fp16=torch.cuda.is_available()
        )
    else:
        args = TrainingArguments(
            output_dir=output_path,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            logging_dir=os.path.join(output_path, "logs"),
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=2,
            load_best_model_at_end=True,
            save_total_limit=1,
            metric_for_best_model="accuracy",
            fp16=torch.cuda.is_available()
        )

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        return {"accuracy": accuracy_score(labels, preds)}

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    )

    trainer.train()
    model.save_pretrained(output_path)
    tokenizer.save_pretrained(output_path)
    print(f"\n✅ {model_name} Evaluation Complete")
    print(trainer.evaluate())

    preds = trainer.predict(test_ds)
    pred_labels = np.argmax(preds.predictions, axis=1)
    print(classification_report(y_test, pred_labels))

In [15]:
# ====== [8] Run DistilBERT ======
train_transformer(
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast,
    "distilbert-base-uncased",
    train_df["text"],
    test_df["text"],
    y_train,
    y_test
)


🚀 Training distilbert-base-uncased...


Map: 100%|██████████| 2104/2104 [00:00<00:00, 14314.66 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\12601\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.247100,0.214560,0.934886
2,0.115300,0.245335,0.942015



✅ distilbert-base-uncased Evaluation Complete


{'eval_loss': 0.24533484876155853, 'eval_accuracy': 0.9420152091254753, 'eval_runtime': 23.5182, 'eval_samples_per_second': 89.463, 'eval_steps_per_second': 11.183, 'epoch': 2.0}
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1001
           1       0.95      0.94      0.94      1103

    accuracy                           0.94      2104
   macro avg       0.94      0.94      0.94      2104
weighted avg       0.94      0.94      0.94      2104



In [17]:
# ====== [9] Run BERT BASE ======
train_transformer(
    BertForSequenceClassification,
    BertTokenizerFast,
    "bert-base-uncased",
    train_df["text"],
    test_df["text"],
    y_train,
    y_test
)


🚀 Training bert-base-uncased...


Map: 100%|██████████| 2104/2104 [00:00<00:00, 11089.07 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\12601\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.242400,0.232237,0.935837
2,0.121400,0.248122,0.944392



✅ bert-base-uncased Evaluation Complete


{'eval_loss': 0.24812234938144684, 'eval_accuracy': 0.9443916349809885, 'eval_runtime': 38.9599, 'eval_samples_per_second': 54.004, 'eval_steps_per_second': 6.751, 'epoch': 2.0}
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1001
           1       0.96      0.94      0.95      1103

    accuracy                           0.94      2104
   macro avg       0.94      0.94      0.94      2104
weighted avg       0.94      0.94      0.94      2104



In [14]:
# ====== [10] Run BERT LARGE ======
train_transformer(
    BertForSequenceClassification,
    BertTokenizerFast,
    "bert-large-uncased",
    train_df["text"],
    test_df["text"],
    y_train,
    y_test
)


🚀 Training bert-large-uncased...


Map: 100%|██████████| 2104/2104 [00:00<00:00, 12632.22 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.111500,0.198201,0.947719



✅ bert-large-uncased Evaluation Complete


{'eval_loss': 0.1982010304927826, 'eval_accuracy': 0.9477186311787072, 'eval_runtime': 333.0969, 'eval_samples_per_second': 6.316, 'eval_steps_per_second': 1.579, 'epoch': 1.9982889269127353}
              precision    recall  f1-score   support

           0       0.94      0.95      0.95      1001
           1       0.96      0.94      0.95      1103

    accuracy                           0.95      2104
   macro avg       0.95      0.95      0.95      2104
weighted avg       0.95      0.95      0.95      2104



In [16]:
# ====== [11] Run RoBERTa (Optional) ======
# Uncomment when your internet connection is stable
train_transformer(
    RobertaForSequenceClassification,
    RobertaTokenizerFast,
   "roberta-base",
    train_df["text"],
    test_df["text"],
    y_train,
    y_test
)


🚀 Training roberta-base...


Map: 100%|██████████| 2104/2104 [00:00<00:00, 14640.70 examples/s]
c:\Users\12601\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\12601\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of RobertaForSequenceC

Epoch,Training Loss,Validation Loss,Accuracy
1,0.293000,0.340182,0.918726
2,0.178400,0.224493,0.944867



✅ roberta-base Evaluation Complete


{'eval_loss': 0.2244928628206253, 'eval_accuracy': 0.9448669201520913, 'eval_runtime': 25.7646, 'eval_samples_per_second': 81.663, 'eval_steps_per_second': 10.208, 'epoch': 2.0}
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      1001
           1       0.96      0.94      0.95      1103

    accuracy                           0.94      2104
   macro avg       0.94      0.95      0.94      2104
weighted avg       0.95      0.94      0.94      2104



In [1]:
import torch
import joblib
import re
import string
import emoji
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import nltk

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

# ===== Preprocessing Function =====
stop_words = set(stopwords.words("english"))
stop_words.difference_update({"no", "not", "don", "shouldn", "wasn", "mustn"})
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

emoticon_dict = {
    ":)": "smiley", ":-)": "smiley", ":D": "laugh", ":-D": "laugh",
    ":(": "sad", ":-(": "sad", ":'(": "cry", ":'-(": "cry",
    ":P": "playful", ":-P": "playful", ";)": "wink", ";-)": "wink",
    ":/": "skeptical", ":-/": "skeptical", ":|": "neutral", ":-|": "neutral",
    ":O": "surprised", ":-O": "surprised", "XD": "laugh", "<3": "love",
    ">:(": "angry", "D:": "horrified", ":-*": "kiss", ":3": "cute",
    ":-X": "sealed lips", "B-)": "cool", "O:)": "angel", ">:)": "evil smile"
}

def preprocess(text):
    text = text.lower()
    text = re.sub(r"http\\S+", "link", text)
    text = re.sub(r"@\\w+", "user_mention", text)
    text = re.sub(r"#(\\w+)", lambda m: m.group(1).replace("_", " "), text)
    text = re.sub(r"\\d+", "", text)

    for emot, meaning in emoticon_dict.items():
        text = text.replace(emot, f" {meaning} ")

    text = emoji.demojize(text).replace(":", "").replace("_", " ")
    text = re.sub(rf"[{string.punctuation}]", "", text)
    text = re.sub(r"(.)\\1{2,}", r"\\1\\1", text)

    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words]
    tokens = [stemmer.stem(lemmatizer.lemmatize(w)) for w in tokens]
    return " ".join(tokens)

# ===== Load Model Function =====
def predict_sentiment(texts, model_type="distilbert"):
    results = []
    if model_type in ["nb_count", "lr_count", "svm_count"]:
        model, vectorizer = joblib.load(f"C:/Users/12601/OneDrive/Desktop/collage works/mini prj/code/saved_models/traditional/{model_type}_bi.joblib")
        for text in texts:
            text_clean = preprocess(text)
            vec = vectorizer.transform([text_clean])
            pred = model.predict(vec)[0]
            results.append("positive" if pred == 1 else "negative")
    else:
        model_path = f"D:/saved_models/transformers/{model_type}"
        tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_path, local_files_only=True)
        model.eval()
        for text in texts:
            with torch.no_grad():
                inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
                outputs = model(**inputs)
                pred = torch.argmax(outputs.logits, dim=1).item()
                results.append("positive" if pred == 1 else "negative")
    return results

# ===== Interactive Input =====
available_models = ["nb_count", "lr_count", "svm_count", "distilbert-base-uncased", "bert-base-uncased"]  # add more if saved
print("Available models:", ", ".join(available_models))
model_choice = input("Enter the model to use: ").strip()

num_inputs = int(input("How many texts do you want to analyze? "))
texts = [input(f"Text {i+1}: ") for i in range(num_inputs)]

# Normalize Hugging Face model name if needed
hf_model_map = {
    "distilbert-base-uncased": "distilbert-base-uncased",
    "bert-base-uncased": "bert-base-uncased"
}
model_key = hf_model_map.get(model_choice, model_choice)

# Predict
sentiments = predict_sentiment(texts, model_key)

# Output
for i, (text, sentiment) in enumerate(zip(texts, sentiments), 1):
    print(f"\nText {i}: {text}\n→ Sentiment: {sentiment}")

c:\Users\12601\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\12601\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\12601\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\12601\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Available models: nb_count, lr_count, svm_count, distilbert-base-uncased, bert-base-uncased

Text 1: tomorrow college is holiday
→ Sentiment: negative

Text 2: the class got cancelled and students got time to play
→ Sentiment: negative


In [10]:
import torch
import joblib
import re
import string
import emoji
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import nltk

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

# ===== Preprocessing =====
stop_words = set(stopwords.words("english"))
stop_words.difference_update({"no", "not", "don", "shouldn", "wasn", "mustn"})
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

emoticon_dict = {
    ":)": "smiley", ":-)": "smiley", ":D": "laugh", ":-D": "laugh",
    ":(": "sad", ":-(": "sad", ":'(": "cry", ":'-(": "cry",
    ":P": "playful", ":-P": "playful", ";)": "wink", ";-)": "wink",
    ":/": "skeptical", ":-/": "skeptical", ":|": "neutral", ":-|": "neutral",
    ":O": "surprised", ":-O": "surprised", "XD": "laugh", "<3": "love",
    ">:(": "angry", "D:": "horrified", ":-*": "kiss", ":3": "cute",
    ":-X": "sealed lips", "B-)": "cool", "O:)": "angel", ">:)": "evil smile"
}

def preprocess(text):
    text = text.lower()
    text = re.sub(r"http\S+", "link", text)
    text = re.sub(r"@\w+", "user_mention", text)
    text = re.sub(r"#(\w+)", lambda m: m.group(1).replace("_", " "), text)
    text = re.sub(r"\d+", "", text)

    for emot, meaning in emoticon_dict.items():
        text = text.replace(emot, f" {meaning} ")

    text = emoji.demojize(text).replace(":", "").replace("_", " ")
    text = re.sub(rf"[{string.punctuation}]", "", text)
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)

    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words]
    tokens = [stemmer.stem(lemmatizer.lemmatize(w)) for w in tokens]
    return " ".join(tokens)

# ===== Prediction =====
def predict_sentiment(texts, model_type):
    results = []

    # Traditional Models
    if model_type in ["nb_count", "lr_count", "svm_count"]:
        model, vectorizer = joblib.load(f"C:/Users/12601/OneDrive/Desktop/collage works/mini prj/code/saved_models/traditional/{model_type}_bi.joblib")
        for text in texts:
            text_clean = preprocess(text)
            vec = vectorizer.transform([text_clean])
            pred = model.predict(vec)[0]
            results.append("positive" if pred == 1 else "negative")

    # Transformer Models
    else:
        model_path = f"D:/saved_models/transformers/{model_type}"
        tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_path, local_files_only=True)
        model.eval()
        for text in texts:
            with torch.no_grad():
                inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
                outputs = model(**inputs)
                pred = torch.argmax(outputs.logits, dim=1).item()
                results.append("positive" if pred == 1 else "negative")

    return results

# ===== User Interaction =====
all_models = {
    "Naive Bayes (CountVec)": "nb_count",
    "Logistic Regression (CountVec)": "lr_count",
    "SVM (CountVec)": "svm_count",
    "DistilBERT": "distilbert-base-uncased",
    "BERT Base": "bert-base-uncased",
    "BERT Large": "bert-large-uncased",
    "RoBERTa Base": "roberta-base"
}

print("\nAvailable Models:")
for i, name in enumerate(all_models, 1):
    print(f"{i}. {name}")

choice = int(input("\nSelect a model by number: "))
model_key = list(all_models.values())[choice - 1]

num_texts = int(input("How many texts would you like to analyze? "))
texts = [input(f"Enter text {i+1}: ") for i in range(num_texts)]

sentiments = predict_sentiment(texts, model_key)

print("\n=== Predictions ===")
for i, (text, sentiment) in enumerate(zip(texts, sentiments), 1):
    print(f"{i}. {text} → {sentiment}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\12601\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\12601\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\12601\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Available Models:
1. Naive Bayes (CountVec)
2. Logistic Regression (CountVec)
3. SVM (CountVec)
4. DistilBERT
5. BERT Base
6. BERT Large
7. RoBERTa Base

=== Predictions ===
1. i can't → positive
2. his grandmother passed away  → negative


In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.save_pretrained(f"D:/saved_models/transformers/{model_name}")
tokenizer.save_pretrained(f"D:/saved_models/transformers/{model_name}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('D:/saved_models/transformers/roberta-base\\tokenizer_config.json',
 'D:/saved_models/transformers/roberta-base\\special_tokens_map.json',
 'D:/saved_models/transformers/roberta-base\\vocab.json',
 'D:/saved_models/transformers/roberta-base\\merges.txt',
 'D:/saved_models/transformers/roberta-base\\added_tokens.json',
 'D:/saved_models/transformers/roberta-base\\tokenizer.json')